<a href="https://colab.research.google.com/github/nblaustein/Ds-2002-Data-Project-1/blob/main/Ds_2002_Project_One.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Load in the NFL Standings xlsx datafile and convert to a csv file
import pandas as pd
df = pd.read_excel('Standings.xlsx')
# Convert the DataFrame to a CSV file
df.to_csv('Standings.csv', index=False)  # Set index=False to exclude the DataFrame index


Loading in the NFL datasets, creating our SQL Database, and creating tables. Our datasets include information about NFL team records and attendance for the 2019 season. We will call a weather api to access information about the temperature and weather conditions on selected game days during the season.

In [2]:

import sqlite3
import csv
# CSV file containing standings data
csv_file = 'Standings.csv'

# Create an SQLite database and connect to it
conn = sqlite3.connect('sports_database.db')
cursor = conn.cursor()

# Define the table schema and create the "standings" table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Standings (
        team name TEXT PRIMARY KEY,
        year INTEGER,
        wins INTEGER,
        losses INTEGER,
        playoffs TEXT
    )
''')

# Read data from the CSV file and insert it into the "standings" table for year 2019
with open(csv_file, 'r') as csv_data:
    csv_reader = csv.DictReader(csv_data)
    for row in csv_reader:
        if row['year'] == '2019':
            cursor.execute('''
                INSERT OR IGNORE INTO standings (team, year, wins, losses, playoffs)
                VALUES (?, ?, ?, ?, ?)
            ''', (row['team_name'], int(row['year']), int(row['wins']), int(row['loss']), row['playoffs']))
conn.commit()

In [3]:
# Viewing the information in the standings table
cursor.execute('SELECT * FROM standings')
data = cursor.fetchall()
for row in data:
    print(row)

('Patriots', 2019, 12, 4, 'Playoffs')
('Bills', 2019, 10, 6, 'Playoffs')
('Jets', 2019, 7, 9, 'No Playoffs')
('Dolphins', 2019, 5, 11, 'No Playoffs')
('Ravens', 2019, 14, 2, 'Playoffs')
('Steelers', 2019, 8, 8, 'No Playoffs')
('Browns', 2019, 6, 10, 'No Playoffs')
('Bengals', 2019, 2, 14, 'No Playoffs')
('Texans', 2019, 10, 6, 'Playoffs')
('Titans', 2019, 9, 7, 'Playoffs')
('Colts', 2019, 7, 9, 'No Playoffs')
('Jaguars', 2019, 6, 10, 'No Playoffs')
('Chiefs', 2019, 12, 4, 'Playoffs')
('Broncos', 2019, 7, 9, 'No Playoffs')
('Raiders', 2019, 7, 9, 'No Playoffs')
('Chargers', 2019, 5, 11, 'No Playoffs')
('Eagles', 2019, 9, 7, 'Playoffs')
('Cowboys', 2019, 8, 8, 'No Playoffs')
('Giants', 2019, 4, 12, 'No Playoffs')
('Redskins', 2019, 3, 13, 'No Playoffs')
('Packers', 2019, 13, 3, 'Playoffs')
('Vikings', 2019, 10, 6, 'Playoffs')
('Bears', 2019, 8, 8, 'No Playoffs')
('Lions', 2019, 3, 12, 'No Playoffs')
('Saints', 2019, 13, 3, 'Playoffs')
('Falcons', 2019, 7, 9, 'No Playoffs')
('Buccaneers',

In [4]:
#do the same thing for the Games csv (containing attendance information)
#convert xlsx to csv file
df = pd.read_excel('Games F.xlsx')
df.rename(columns={"City of Stadium": "City"}, inplace=True)
df.rename(columns={"home_team": "HomeTeam"}, inplace=True)
# Convert the DataFrame to a CSV file
df.to_csv('Games.csv', index=False)  # Set index=False to exclude the DataFrame index


In [5]:
import sqlite3
#Create the Attendance table
conn = sqlite3.connect('sports_database.db')
cursor = conn.cursor()
# Create the "Attendance" table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Attendance (
        Week INTEGER,
        Date TEXT,
        HomeTeam TEXT,
        Attendance INTEGER,
        City TEXT
    )
''')
conn.commit()

In [6]:

csv_file = 'Games.csv'
# Read data from the CSV file and insert it into the "attendance" table
with open(csv_file, 'r') as csv_data:
    csv_reader = csv.DictReader(csv_data)
    for row in csv_reader:
        cursor.execute('''
            INSERT INTO Attendance (Week, Date, HomeTeam, Attendance, City)
            VALUES (?, ?, ?, ?,?)
        ''', (int(row['week']), row['date'], row['HomeTeam'], row['Attendance'], row['City']))
conn.commit()

In [7]:
#View the information in the Attendance Table
import sqlite3
conn = sqlite3.connect('sports_database.db')
cursor = conn.cursor()
cursor = conn.cursor()
cursor.execute('SELECT * FROM Attendance')
data = cursor.fetchall()
for row in data:
    print(row)

(1, '2019-09-08', 'Arizona Cardinals', 60687, 'Glendale')
(1, '2019-09-08', 'Carolina Panthers', 72005, 'Charlotte')
(1, '2019-09-05', 'Chicago Bears', 62435, 'Chicago')
(1, '2019-09-08', 'Cleveland Browns', 67431, 'Cleveland')
(1, '2019-09-08', 'Dallas Cowboys', 90353, 'Arlington')
(1, '2019-09-08', 'Jacksonville Jaguars', 60157, 'Jacksonville')
(1, '2019-09-08', 'Los Angeles Chargers', 25363, 'Los Angeles')
(1, '2019-09-08', 'Miami Dolphins', 65012, 'Miami')
(1, '2019-09-08', 'Minnesota Vikings', 66714, 'Minneapolis')
(1, '2019-09-08', 'New England Patriots', 65878, 'Foxborough')
(1, '2019-09-09', 'New Orleans Saints', 73039, 'New Orleans')
(1, '2019-09-08', 'New York Jets', 78523, 'East Rutherford')
(1, '2019-09-09', 'Oakland Raiders', 52359, 'Oakland')
(1, '2019-09-08', 'Philadelphia Eagles', 69696, 'Philadelphia')
(1, '2019-09-08', 'Seattle Seahawks', 68710, 'Seattle')
(1, '2019-09-08', 'Tampa Bay Buccaneers', 55976, 'Tampa')
(2, '2019-09-15', 'Atlanta Falcons', 72443, 'Atlanta')


In [8]:
#load in the weather api and create a dataframe that we will then convert into a sql table
import urllib.request
import csv
import pandas as pd

# Create an empty Pandas DataFrame to store the data
columns = ['Location', 'Date', 'Datetime', 'Name', 'Temperature', 'Precipitation', 'Snow', 'Visibility', 'Conditions', 'Description']
df = pd.DataFrame(columns=columns)

# Define a list of city-date pairs needed for our queries (based on game dates and location of stadiums)
# Define a list of city-date pairs
city_date_pairs = [
('chicago', '2019-09-05'),
('philadelphia', '2019-09-05'),
('cleveland', '2019-09-08'),
('east%20rutherford', '2019-09-08'),
('miami', '2019-09-08'),
('minneapolis', '2019-09-08'),
('new%20orleans', '2019-09-09'),
('oakland', '2019-09-09'),
('charlotte', '2019-09-12'),
('cincinnati', '2019-09-15'),
('east%20rutherford', '2019-09-15'),
('landover', '2019-09-15'),
('los%20angeles', '2019-09-15'),
('pittsburgh', '2019-09-15'),
('santa%20clara', '2019-09-15'),
('seattle', '2019-09-15'),
('tampa', '2019-09-15'),
('glendale', '2019-09-15'),
('jacksonville', '2019-09-15'),
('kansas%20city', '2019-09-15'),
('denver', '2019-09-15'),
('atlanta', '2019-09-15'),
('baltimore', '2019-09-15'),
('charlotte', '2019-09-12'),
('cincinnati', '2019-09-15'),
('denver', '2019-09-15'),
('detroit', '2019-09-15'),
('green%20bay', '2019-09-15'),
('houston', '2019-09-15'),
('los%20angeles', '2019-09-15'),
('miami%20gardens', '2019-09-15'),
('east%20rutherford', '2019-09-15'),
('east%20rutherford', '2019-09-16'),
('oakland', '2019-09-15'),
('pittsburgh', '2019-09-15'),
('nashville', '2019-09-15'),
('landover', '2019-09-15'),
('glendale', '2019-09-22'),
('buffalo', '2019-09-22'),
('cleveland', '2019-09-22'),
('arlington%20texas', '2019-09-22'),
('green%20bay', '2019-09-22'),
('indianapolis', '2019-09-22'),
('jacksonville', '2019-09-19'),
('kansas%20city', '2019-09-22'),
('carson', '2019-09-22'),
('minneapolis', '2019-09-22'),
('fox%20borough', '2019-09-22'),
('philadelphia', '2019-09-22'),
('santa%20clara', '2019-09-22'),
('seattle', '2019-09-22'),
('tampa', '2019-09-22'),
('landover', '2019-09-23'),
('glendale', '2019-12-15'),
('baltimore', '2019-12-12'),
('charlotte', '2019-12-15'),
('cincinnati', '2019-12-15'),
('arlington%20texas', '2019-12-15'),
('detroit', '2019-12-15'),
('green%20bay', '2019-12-15'),
('kansas%20city', '2019-12-15'),
('carson', '2019-12-15'),
('new%20orleans', '2019-12-16'),
('east%20rutherford', '2019-12-15'),
('oakland', '2019-12-15'),
('pittsburgh', '2019-12-15'),
('santa%20clara', '2019-12-15'),
('nashville', '2019-12-15'),
('landover', '2019-12-15'),
('atlanta', '2019-12-22'),
('chicago', '2019-12-22'),
('cleveland', '2019-12-22'),
('denver', '2019-12-22'),
('indianapolis', '2019-12-22'),
('carson', '2019-12-22'),
('miami%20gardens', '2019-12-22'),
('minneapolis', '2019-12-23'),
('fox%20borough', '2019-12-21'),
('east%20rutherford', '2019-12-22'),
('philadelphia', '2019-12-22'),
('santa%20clara', '2019-12-21'),
('seattle', '2019-12-22'),
('tampa', '2019-12-21'),
('nashville', '2019-12-22'),
('baltimore', '2019-12-29'),
('buffalo', '2019-12-29'),
('charlotte', '2019-12-29'),
('cincinnati', '2019-12-29'),
('arlington%20texas', '2019-12-29'),
('denver', '2019-12-29'),
('detroit', '2019-12-29'),
('houston', '2019-12-29'),
('jacksonville', '2019-12-29'),
('kansas%20city', '2019-12-29'),
('los%20angeles', '2019-12-29'),
('minneapolis', '2019-12-29'),
('fox%20borough', '2019-12-29'),
('east%20rutherford', '2019-12-29'),
('seattle', '2019-12-29'),
('tampa', '2019-12-29')
]
try:
    for city, date in city_date_pairs:
        # Construct the API URL for the current location and date
        #url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}/{date}/{date}?unitGroup=metric&elements=datetime%2Cname%2Ctemp%2Cprecip%2Csnow%2Cvisibility%2Cconditions%2Cdescription&include=days&key=PW34YGHTRKDZV4P34V5MJVQQG&contentType=csv"
        url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}/{date}/{date}?unitGroup=metric&elements=datetime%2Cname%2Ctemp%2Cprecip%2Csnow%2Cvisibility%2Cconditions%2Cdescription&include=days&key=DDVY9RNENV3WL6DTAZ4GXE2FE&contentType=csv"
        # Fetch the CSV data from the URL
        with urllib.request.urlopen(url) as response:
            data = response.read().decode('utf-8')

        # Parse the data as CSV
        CSVText = csv.reader(data.splitlines())

        # Skip the first row (column headers)
        next(CSVText)

        # Create a list to hold the data
        data_list = []
        for row in CSVText:
            data_list.append([city, date] + row)

        # Append the data to the Pandas DataFrame
        temp_df = pd.DataFrame(data_list, columns=columns)
        df = df.append(temp_df, ignore_index=True)

except urllib.error.HTTPError as e:
    ErrorInfo = e.read().decode()
    print('Error code:', e.code, ErrorInfo)
except urllib.error.URLError as e:
    ErrorInfo = e.reason
    print('Error:', ErrorInfo)

# Drop the first column (Location) from the DataFrame
df = df.iloc[:, 1:]




<ipython-input-8-148583e936b6>:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
<ipython-input-8-148583e936b6>:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
<ipython-input-8-148583e936b6>:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
<ipython-input-8-148583e936b6>:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
<ipython-input-8-148583e936b6>:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Us

In [9]:
#Renaming the columns in the weather dataframe
new_column_names = ['Date', 'Location','Datetime', 'Temperature', 'Precipitation', 'Snow', 'Visibility', 'Conditions', 'Description']
weather_table = df.rename(columns=dict(zip(df.columns, new_column_names)))


In [11]:
#convert weather dataframe to a sql table
conn = sqlite3.connect('sports_database.db')
# Use the to_sql method to write the DataFrame to an SQL table
weather_table.to_sql('weather_table', conn, if_exists='replace', index=False)

100

In [12]:
#select the information from the weather table
cursor = conn.cursor()
cursor.execute('SELECT * FROM weather_table')
data = cursor.fetchall()
for row in data:
    print(row)


('2019-09-05', 'Chicago, IL, United States', '2019-09-05', '18.3', '0', '0', '16', 'Partially cloudy', 'Clearing in the afternoon.')
('2019-09-05', 'Philadelphia, PA, United States', '2019-09-05', '21.9', '0', '0', '16', 'Partially cloudy', 'Partly cloudy throughout the day.')
('2019-09-08', 'Cleveland, OH, United States', '2019-09-08', '20.2', '0', '0', '16', 'Partially cloudy', 'Partly cloudy throughout the day.')
('2019-09-08', 'East Rutherford, NJ, United States', '2019-09-08', '20.9', '0', '0', '16', 'Partially cloudy', 'Partly cloudy throughout the day.')
('2019-09-08', 'Miami, FL, United States', '2019-09-08', '29.9', '0', '', '16', 'Partially cloudy', 'Becoming cloudy in the afternoon.')
('2019-09-08', 'Minneapolis, MN, United States', '2019-09-08', '13.9', '4.044', '0', '15.7', 'Rain, Partially cloudy', 'Partly cloudy throughout the day with a chance of rain throughout the day.')
('2019-09-09', 'New Orleans, LA, United States', '2019-09-09', '32', '0', '', '15.9', 'Partially c

In [13]:
#Create the Fact Table

# Connect to the SQLite database
conn = sqlite3.connect('sports_database.db')
cursor = conn.cursor()
# Define the table schema for the fact table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS NFL_Fact_Table (
        TeamName TEXT PRIMARY KEY,
        Wins INTEGER,
        Losses INTEGER,
        PlayoffResult TEXT

    )
''')

# Write an SQL query to create the fact table by joining info from the standings and attendance tables
query = '''

INSERT OR REPLACE INTO NFL_Fact_Table (TeamName, Wins, Losses, PlayoffResult)
SELECT
    s.team AS TeamName,
    s.wins AS Wins,
    s.losses AS Losses,
    s.playoffs AS PlayoffResult
FROM
    standings AS s
JOIN
    Attendance AS a
ON
    a.HomeTeam LIKE '%' || s.team || '%';
'''

# Execute the query to populate the fact table
cursor.execute(query)
conn.commit()


In [14]:
#View fact table inserts
# Execute an SQL query to select data from the standings table
cursor.execute('SELECT * FROM NFL_Fact_Table')
data = cursor.fetchall()
for row in data:
    print(row)

('Patriots', 12, 4, 'Playoffs')
('Bills', 10, 6, 'Playoffs')
('Jets', 7, 9, 'No Playoffs')
('Dolphins', 5, 11, 'No Playoffs')
('Ravens', 14, 2, 'Playoffs')
('Steelers', 8, 8, 'No Playoffs')
('Browns', 6, 10, 'No Playoffs')
('Bengals', 2, 14, 'No Playoffs')
('Texans', 10, 6, 'Playoffs')
('Titans', 9, 7, 'Playoffs')
('Colts', 7, 9, 'No Playoffs')
('Jaguars', 6, 10, 'No Playoffs')
('Chiefs', 12, 4, 'Playoffs')
('Broncos', 7, 9, 'No Playoffs')
('Raiders', 7, 9, 'No Playoffs')
('Chargers', 5, 11, 'No Playoffs')
('Eagles', 9, 7, 'Playoffs')
('Cowboys', 8, 8, 'No Playoffs')
('Giants', 4, 12, 'No Playoffs')
('Redskins', 3, 13, 'No Playoffs')
('Packers', 13, 3, 'Playoffs')
('Vikings', 10, 6, 'Playoffs')
('Bears', 8, 8, 'No Playoffs')
('Lions', 3, 12, 'No Playoffs')
('Saints', 13, 3, 'Playoffs')
('Falcons', 7, 9, 'No Playoffs')
('Buccaneers', 7, 9, 'No Playoffs')
('Panthers', 5, 11, 'No Playoffs')
('49ers', 13, 3, 'Playoffs')
('Seahawks', 11, 5, 'Playoffs')
('Rams', 9, 7, 'No Playoffs')
('Cardin

In [15]:
#View the columns in all of the tables
conn = sqlite3.connect('sports_database.db')
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(standings)")
columns = cursor.fetchall()
column_names = [column[1] for column in columns]
print("Column Names in Standings Table:", column_names)

cursor.execute("PRAGMA table_info(weather_table)")
columns = cursor.fetchall()
column_names = [column[1] for column in columns]
print("Column Names in Weather Table:", column_names)

cursor.execute("PRAGMA table_info(NFL_Fact_Table)")
columns = cursor.fetchall()
column_names = [column[1] for column in columns]
print("Column Names in Fact Table:", column_names)

cursor.execute("PRAGMA table_info(Attendance)")
columns = cursor.fetchall()
column_names = [column[1] for column in columns]
print("Column Names in Attendance Table:", column_names)

Column Names in Standings Table: ['team', 'year', 'wins', 'losses', 'playoffs']
Column Names in Weather Table: ['Date', 'Location', 'Datetime', 'Temperature', 'Precipitation', 'Snow', 'Visibility', 'Conditions', 'Description']
Column Names in Fact Table: ['TeamName', 'Wins', 'Losses', 'PlayoffResult']
Column Names in Attendance Table: ['Week', 'Date', 'HomeTeam', 'Attendance', 'City']


**Query Number One - Average Home Game Attendance and Total Team Wins**

In [16]:
#Creating a new column called AverageAttendance in the NFL_Fact_Table
## Note: This code only needs to be run the first time (once the table is created, this code can be commented out)
import sqlite3
# Connect to the SQLite database
conn = sqlite3.connect('sports_database.db')
cursor = conn.cursor()

# Step 1: Calculate the average attendance for each game
query = '''
     SELECT Date, SUM(Attendance) / 13 AS AverageAttendance
     FROM Attendance
     GROUP BY Date;
'''

cursor.execute(query)
average_attendance_data = cursor.fetchall()

# Step 2: Add a new column for AverageAttendance in the NFL_Fact_Table
alter_table_query = '''
ALTER TABLE NFL_Fact_Table
ADD COLUMN AverageAttendance REAL;
'''
cursor.execute(alter_table_query)

In [17]:
# Query 1: Determining total wins and average attendance for each team for 2019 season home games
## insert average attendance into the fact table to do this.

# Step 3: Update the NFL_Fact_Table with the calculated average attendance
update_query = '''
UPDATE NFL_Fact_Table
SET AverageAttendance = (
    SELECT SUM(Attendance) / 8
    FROM Attendance
    WHERE Attendance.HomeTeam LIKE '%' || NFL_Fact_Table.TeamName || '%'
)
'''
cursor.execute(update_query)

# Commit the changes to the database
conn.commit()


In [18]:
#View information from the Fact Table (now including average attendance across home games)
cursor.execute('SELECT * FROM NFL_Fact_Table')
data = cursor.fetchall()
for row in data:
    print(row)

('Patriots', 12, 4, 'Playoffs', 68255.0)
('Bills', 10, 6, 'Playoffs', 68154.0)
('Jets', 7, 9, 'No Playoffs', 68650.0)
('Dolphins', 5, 11, 'No Playoffs', 69652.0)
('Ravens', 14, 2, 'Playoffs', 71253.0)
('Steelers', 8, 8, 'No Playoffs', 64632.0)
('Browns', 6, 10, 'No Playoffs', 63459.0)
('Bengals', 2, 14, 'No Playoffs', 68948.0)
('Texans', 10, 6, 'Playoffs', 65699.0)
('Titans', 9, 7, 'Playoffs', 60776.0)
('Colts', 7, 9, 'No Playoffs', 65985.0)
('Jaguars', 6, 10, 'No Playoffs', 60684.0)
('Chiefs', 12, 4, 'Playoffs', 65125.0)
('Broncos', 7, 9, 'No Playoffs', 68267.0)
('Raiders', 7, 9, 'No Playoffs', 60651.0)
('Chargers', 5, 11, 'No Playoffs', 54380.0)
('Eagles', 9, 7, 'Playoffs', 68347.0)
('Cowboys', 8, 8, 'No Playoffs', 81547.0)
('Giants', 4, 12, 'No Playoffs', 67965.0)
('Redskins', 3, 13, 'No Playoffs', 62908.0)
('Packers', 13, 3, 'Playoffs', 70113.0)
('Vikings', 10, 6, 'Playoffs', 65482.0)
('Bears', 8, 8, 'No Playoffs', 63228.0)
('Lions', 3, 12, 'No Playoffs', 65062.0)
('Saints', 13, 3,

**Question 1:**
Is there an association between an NFL team's record and the average attendance they had across all home games in the 2019 season?

In [19]:
### Doing a query from the fact table to get the average attendance and total wins
import sqlite3
conn = sqlite3.connect('sports_database.db')
cursor = conn.cursor()

# Define the SQL query to select average attendance and total wins
query = '''
SELECT
    TeamName,
    AverageAttendance,
    Wins
FROM NFL_Fact_Table
ORDER BY Wins DESC;
'''
# Execute the query
cursor.execute(query)
# Fetch the results
results = cursor.fetchall()
# Print the results
for row in results:
    print(f"Team: {row[0]}, Average Attendance: {row[1]}, Total Wins: {row[2]}")


Team: Ravens, Average Attendance: 71253.0, Total Wins: 14
Team: Packers, Average Attendance: 70113.0, Total Wins: 13
Team: Saints, Average Attendance: 67914.0, Total Wins: 13
Team: 49ers, Average Attendance: 64760.0, Total Wins: 13
Team: Patriots, Average Attendance: 68255.0, Total Wins: 12
Team: Chiefs, Average Attendance: 65125.0, Total Wins: 12
Team: Seahawks, Average Attendance: 66914.0, Total Wins: 11
Team: Bills, Average Attendance: 68154.0, Total Wins: 10
Team: Texans, Average Attendance: 65699.0, Total Wins: 10
Team: Vikings, Average Attendance: 65482.0, Total Wins: 10
Team: Titans, Average Attendance: 60776.0, Total Wins: 9
Team: Eagles, Average Attendance: 68347.0, Total Wins: 9
Team: Rams, Average Attendance: 67861.0, Total Wins: 9
Team: Steelers, Average Attendance: 64632.0, Total Wins: 8
Team: Cowboys, Average Attendance: 81547.0, Total Wins: 8
Team: Bears, Average Attendance: 63228.0, Total Wins: 8
Team: Jets, Average Attendance: 68650.0, Total Wins: 7
Team: Colts, Averag

**Conclusion:**
There does appear to be some sort of correlation between a team's total wins for the season and their average attendance for all of the home games in the 2019 season. The Ravens and Packers had the most and second most amount of wins and the third and fourth highest average attendances, respectively. There is somewhat of a downward trend after that with average attendance as wins decreases, however, there are some outliers such as the Cowboys who only had 8 wins, but had the highest average attendance with 81,547. Of course other factors impact attendance as well, but this is interesting data to visualize.

**Query Number Two - Average Temperature (In Celsius) and Average Attendance during the first and last three weeks of the NFL 2019 season**

**Question 2: Is there an association between average temperature (in Celsius) and the average home game attendance for NFL teams in the first and last three weeks of the 2019 season?**


In [20]:
#Using the weather_table, temperature is pulled from the weather table and attendance for games in the first
#and last three weeks of the season is pulled from the Attendance table. Then python code is used to get average temperatures
# of each location and get average attendance for those selected games as well.

import sqlite3
# Connect to the database
conn = sqlite3.connect('sports_database.db')
cursor = conn.cursor()

# Execute the SQL query with aggregation
cursor.execute('''
    SELECT w.Location, AVG(w.Temperature) AS AvgTemperature, AVG(a.Attendance) AS AvgAttendance
    FROM weather_table AS w
    JOIN Attendance AS a ON w.Date = a.Date
    GROUP BY w.Location
    ORDER BY AvgTemperature DESC;
''')
# There were issues without a dictionary and just made it for convenience
city_totals = {}
# Accumulate totals
for row in cursor.fetchall():
    location, avg_temp, avg_attendance = row
    if location in city_totals:
        city_totals[location][0] += avg_temp
        city_totals[location][1] += avg_attendance
        city_totals[location][2] += 1
    else:
        city_totals[location] = [avg_temp, avg_attendance, 1]
# Calculate averages
for city, (total_temp, total_attendance, count) in city_totals.items():
    avg_temp = total_temp / count
    avg_attendance = total_attendance / count
    print(f"Location: {city}, Average Temperature: {avg_temp}, Average Attendance: {avg_attendance}")
conn.close()

Location: Miami, FL, United States, Average Temperature: 29.899999999999995, Average Attendance: 65115.769230769234
Location: New Orleans, LA, United States, Average Temperature: 28.666666666666668, Average Attendance: 66218.33333333333
Location: Miami Gardens, FL, United States, Average Temperature: 26.68461538461539, Average Attendance: 63446.03846153846
Location: Tampa, FL, United States, Average Temperature: 25.248936170212776, Average Attendance: 66758.8085106383
Location: Jacksonville, FL, United States, Average Temperature: 24.525806451612915, Average Attendance: 67164.67741935483
Location: Glendale, AZ, United States, Average Temperature: 24.100000000000016, Average Attendance: 67314.47619047618
Location: Houston, TX, United States, Average Temperature: 23.19333333333333, Average Attendance: 67449.73333333334
Location: Foxborough, Goose Creek, SC, United States, Average Temperature: 19.7090909090909, Average Attendance: 66363.30303030302
Location: Landover, Hyattsville, MD, Uni

**Conclusion:**
Because we only have data for a short period and other variables are involved, it is difficult to see any real trends between the average temperature and average attendance of games in the first and last weeks of the 2019 season and some teams didn't play every week due to scheduled breaks. The locations with the coldest two average temperatures did have relatively low average attendances, but there isn't an overall large trend.


Nicole Blaustein, Kiduse Gezehagne, Rachit Goli